In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric,load_dataset
import evaluate

In [2]:
id2label = {0: "suicide", 1:"non-suicide"}
label2id = {"suicide":0, "non-suicide":1}

In [3]:
tokeniser = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
dataset = load_dataset("vibhorag101/phr_suicide_prediction_dataset")

In [15]:
def tokeniseDataset(dataset):
    return(tokeniser(dataset["text"],padding="max_length",truncation=True))

def convertLabel2ID(dataset):
    dataset['label'] = label2id[dataset['label']]
    return dataset
    
dataset = dataset.map(convertLabel2ID) 
tokenisedDataset = dataset.map(tokeniseDataset,batched=True)

trainTokeniseDataset = tokenisedDataset["train"]
testTokenisedDataset= tokenisedDataset["test"]

Map:   0%|          | 0/185659 [00:00<?, ? examples/s]

Map:   0%|          | 0/46415 [00:00<?, ? examples/s]

In [25]:
def compute_metrics(eval_pred):
    metric_acc = evaluate.load("accuracy")
    metric_rec = evaluate.load("recall")
    metric_pre = evaluate.load("precision")
    metric_f1 = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = metric_acc.compute(predictions=predictions, references=labels)
    recall = metric_rec.compute(predictions=predictions, references=labels)
    precision = metric_pre.compute(predictions=predictions, references=labels)
    f1 = metric_f1.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}


In [26]:
wandb.login()

True

In [27]:
training_args = TrainingArguments(
    output_dir="Model",
    report_to = 'wandb',
    learning_rate=2e-5,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    evaluation_strategy="epoch",
    save_strategy="steps",
    save_steps=1500
)

In [28]:
testDataset = testTokenisedDataset.select(range(1000))
print(testDataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [29]:
trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset=trainTokeniseDataset,
    eval_dataset=testDataset,
    compute_metrics=compute_metrics,
)

In [30]:
predictions = trainer.predict(testDataset)
metrics = predictions.metrics
print(metrics)

{'test_loss': 0.6956313848495483, 'test_accuracy': {'accuracy': 0.484}, 'test_recall': {'recall': 0.8936170212765957}, 'test_precision': {'precision': 0.5005417118093174}, 'test_f1': {'f1': 0.6416666666666666}, 'test_runtime': 26.7241, 'test_samples_per_second': 37.419, 'test_steps_per_second': 37.419}


In [ ]:
trainer.train()